In [ ]:
%pip install -e ..[dev]

In [ ]:
from pprint import pprint
import os
import time
from focoos.focoos import Focoos, DeploymentMode
from dotenv import load_dotenv


load_dotenv()

host_url = "https://api.dev.focoos.ai/v0"
#local_url = "http://localhost:8501/v0"

focoos = Focoos(api_key=os.getenv("FOCOOS_API_KEY"),host_url=host_url)

model = focoos.get_model("c2a157fd006f4a6", deployment_mode=DeploymentMode.REMOTE)
t0 = time.time()
detections = model.infer("./assets/ade_val_034.jpg",threshold=0.5)
t1 = time.time()
print(f"Time taken: {t1-t0} seconds")
pprint(detections)

In [ ]:
focoos.unload_model("c2a157fd006f4a65")

In [ ]:
from pprint import pprint
models = focoos.list_models()
pprint(models)
models = focoos.list_focoos_models()
pprint(models)

In [ ]:
model_info = focoos.get_model_info("focoos_rtdetr")
print(model_info["status"])
pprint(model_info)